In [ ]:
import pandas as pd
import numpy as np
import psycopg2

# Load datasets
moma_artists = pd.read_csv('Artists.csv')
moma_artworks = pd.read_csv('Artworks.csv')
painter_palette = pd.read_csv('PainterPalette.csv')

In [2]:
moma_artists.isna().sum()

ConstituentID        0
DisplayName          0
ArtistBio         2205
Nationality       2499
Gender            3276
BeginDate            0
EndDate              0
Wiki QID         12420
ULAN             12736
dtype: int64

In [3]:
moma_artworks.isna().sum()

Title                     48
Artist                  1269
ConstituentID           1269
ArtistBio               6748
Nationality             1269
BeginDate               1269
EndDate                 1269
Gender                  1269
Date                    2021
Medium                  9250
Dimensions              8641
CreditLine              1545
AccessionNumber            0
Classification             0
Department                 0
DateAcquired            6750
Cataloged                  0
ObjectID                   0
URL                    57349
ImageURL               66468
OnView                156577
Circumference (cm)    157741
Depth (cm)            139633
Diameter (cm)         156351
Height (cm)            29384
Length (cm)           157028
Weight (kg)           157454
Width (cm)             30242
Seat Height (cm)      157751
Duration (sec.)       155795
dtype: int64

In [4]:
painter_palette.isna().sum()

artist                           0
Nationality                   7934
citizenship                   3476
gender                         426
styles                        7159
movement                      7159
Art500k_Movements             8013
birth_place                   1565
death_place                   3562
birth_year                     692
death_year                    2606
FirstYear                     3161
LastYear                      3161
wikiart_pictures_count        7159
locations                       93
locations_with_years            93
styles_extended               7159
StylesCount                   7786
StylesYears                   8012
occupations                    255
PaintingsExhibitedAt          6318
PaintingsExhibitedAtCount     6318
PaintingSchool                9440
Influencedby                  9889
Influencedon                 10058
Pupils                       10202
Teachers                     10182
FriendsandCoworkers          10016
Contemporary        

In [5]:
print(len(painter_palette.index))

10361


Clean MOMA Artists dataset:

In [6]:
# Ensure ConstituentID is unique
moma_artists = moma_artists.drop_duplicates(subset=['ConstituentID'])

# Extract birth and death years from ArtistBio. Fill missing values and convert data types.
moma_artists['birth_year'] = moma_artists['ArtistBio'].str.extract(r'(\d{4})–').astype(float).fillna(0).astype(int)
moma_artists['death_year'] = moma_artists['ArtistBio'].str.extract(r'–(\d{4})').astype(float).fillna(0).astype(int)
moma_artists['Nationality'] = moma_artists['Nationality'].fillna('Unknown')
moma_artists['Gender'] = moma_artists['Gender'].fillna('Unknown')

Clean MOMA Artworks dataset:

In [7]:
# Ensure artwork is unique
moma_artworks = moma_artworks.drop_duplicates(subset=['ObjectID'], keep='first')
moma_artworks = moma_artworks.drop_duplicates(subset=['Title'])

# Convert ConstituentID column to string, split, explode, and convert to integers
moma_artworks['ConstituentID'] = moma_artworks['ConstituentID'].astype(str).str.split(', ')
moma_artworks = moma_artworks.explode('ConstituentID')
moma_artworks['ConstituentID'] = pd.to_numeric(moma_artworks['ConstituentID'], errors='coerce').fillna(0).astype(int)

# Filter out rows where ConstituentID is 0
moma_artworks = moma_artworks[moma_artworks['ConstituentID'] != 0]

# Extract start and end years from the date column
moma_artworks['start_year'] = moma_artworks['Date'].str.extract(r'(\d{4})').astype(float)  # Extract start year
moma_artworks['end_year'] = moma_artworks['Date'].str.extract(r'-(\d{2})').astype(float)  # Extract end year (last 2 digits)

# Handle cases where the end year is only 2 digits (e.g., "1976-77")
moma_artworks['start_year'] = moma_artworks['Date'].str.extract(r'(\d{4})').astype(float).fillna(0).astype(int)
moma_artworks['end_year'] = moma_artworks['Date'].str.extract(r'-(\d{2})').astype(float)
moma_artworks['end_year'] = (
    moma_artworks['start_year'].astype(str).str[:2] + 
    moma_artworks['end_year'].astype(str).str.zfill(2)
).fillna(moma_artworks['start_year'].astype(str))

moma_artworks['start_year'] = pd.to_numeric(moma_artworks['start_year'], errors='coerce').fillna(0).astype(int)
moma_artworks['end_year'] = pd.to_numeric(moma_artworks['end_year'], errors='coerce').fillna(0).astype(int)

moma_artworks['Artist'] = moma_artworks['Artist'].fillna('Unknown')

Clean PainterPalette dataset:

In [8]:
# Clean PainterPalette dataset
columns_to_clean = [
    'Nationality', 'citizenship', 'gender', 'styles', 'movement', 'birth_place', 
    'death_place', 'occupations', 'Influencedby', 'Influencedon', 
    'Pupils', 'Teachers', 'FriendsandCoworkers', 'Contemporary', 'PaintingSchool'
]

painter_palette[columns_to_clean] = painter_palette[columns_to_clean].fillna('Unknown')
for col in ['styles', 'Nationality', 'Influencedby', 'Influencedon', 'Pupils', 'Teachers'
            , 'FriendsandCoworkers', 'Contemporary', 'occupations']:
    painter_palette[col] = painter_palette[col].str.split(',')

Use psycopg2 to connect to PostgreSQL DB:

In [9]:
# Merge MOMA Artists and PainterPalette datasets
artists_combined = pd.merge(
    moma_artists, 
    painter_palette, 
    left_on='DisplayName', 
    right_on='artist', 
    how='left'
)

artists_combined['Nationality'] = artists_combined['Nationality_x']
artists_combined = artists_combined.drop(columns=['Nationality_x', 'Nationality_y'])

In [ ]:
# Connect to Postgres
conn = psycopg2.connect(
    dbname="art",
    user="postgres",
    password="",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

In [155]:
# Insert into Artists table
for _, row in artists_combined.iterrows():
    cursor.execute("""
        INSERT INTO Artists (artist_id, display_name, nationality, gender, birth_year, death_year)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['ConstituentID'],
        row['DisplayName'],
        row.get('Nationality', 'Unknown'),  # Use the merged 'Nationality' column
        row.get('Gender', 'Unknown'),
        row.get('birth_year_x', 0),  # Ensure you're using the correct birth year column
        row.get('death_year_x', 0)   # Ensure you're using the correct death year column
    ))

In [156]:
moma_artworks = moma_artworks.drop_duplicates(subset=['ObjectID'], keep='first')

# Insert into Artworks table
for _, row in moma_artworks.iterrows():
    cursor.execute("""
        INSERT INTO Artworks (artwork_id, title, artist_id, artwork_start_date, medium)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        row['ObjectID'],
        row['Title'],
        row['ConstituentID'],
        row['start_year'],
        row['Medium'],
    ))

In [157]:
# Insert into Movements table
unique_movements = pd.DataFrame(painter_palette['movement'].unique(), columns=['movement_name'])
for _, row in unique_movements.iterrows():
    cursor.execute("""
        INSERT INTO Movements (name)
        VALUES (%s)
    """, (row['movement_name'],))

In [158]:
# Insert into Artist_Movements table (mapping artist to their movements)
for _, row in artists_combined.iterrows():
    movement = row['movement']
    # Replace NaN with an empty string
    if pd.isna(movement):
        movement = ''
    if isinstance(movement, str) and movement != '':
        movement_ids = []
        for movement in movement.split(','):
            movement = movement.strip()
            cursor.execute("SELECT movement_id FROM Movements WHERE name = %s", (movement,))
            movement_id = cursor.fetchone()
            if movement_id:
                movement_ids.append(movement_id[0])
        for movement_id in movement_ids:
            cursor.execute("""
                INSERT INTO Artist_Movements (artist_id, movement_id)
                VALUES (%s, %s)
            """, (row['ConstituentID'], movement_id))
    else:
        continue

In [159]:
# Insert into Styles table
unique_styles = pd.DataFrame(painter_palette['styles'].explode().unique(), columns=['style_name'])
for _, row in unique_styles.iterrows():
    cursor.execute("""
        INSERT INTO Styles (name)
        VALUES (%s)
    """, (row['style_name'],))

In [ ]:
# Convert non-list entries into a list
artists_combined['styles'] = artists_combined['styles'].apply(lambda x: x if isinstance(x, list) else [x] if pd.notna(x) else [])

# Iterate through rows and insert into the Artist_Styles table
for _, row in artists_combined.iterrows():
    # Check if the 'styles' column is empty or contains "Unknown" or any NaN
    if not row['styles'] or "Unknown" in row['styles']:
        continue  # Skip this row if it has no styles or "Unknown"
    
    # Iterate through styles and insert into Artist_Styles
    for style in row['styles']:
        if pd.isna(style) or style == "NaN":
            continue  # Skip invalid style values
        
        # Execute query to get the style_id from the Styles table
        cursor.execute("SELECT style_id FROM Styles WHERE name = %s", (style,))
        style_id = cursor.fetchone()
        if style_id:
            style_id = style_id[0]  # Extract style_id
            cursor.execute("""
                INSERT INTO Artist_Styles (artist_id, style_id)
                VALUES (%s, %s)
            """, (row['ConstituentID'], style_id))

In [ ]:
# Combine 'birth_place' and 'death_place' and get unique locations
unique_locations = pd.DataFrame(pd.concat([painter_palette['birth_place'], painter_palette['death_place']]).unique(), columns=['location_name'])

# Insert into Locations table
for _, row in unique_locations.iterrows():
    cursor.execute("""
        INSERT INTO Locations (name)
        VALUES (%s)
    """, (row['location_name'],))

In [ ]:
# Insert into Artist_Locations table
for _, row in artists_combined.iterrows():
    location_ids = []
    # Replace NaNs with empty string
    for location in [row['birth_place'], row['death_place']]:
        if pd.isna(location):
            location = ''
        
        # Check if location exists in Locations table
        cursor.execute("SELECT location_id FROM Locations WHERE name = %s", (location,))
        location_id = cursor.fetchone()
        
        # If location_id is found, append it to location_ids list
        if location_id:
            location_ids.append(location_id[0])
    
    # Insert only if combination of artist_id and location_id doesn't already exist
    for location_id in location_ids:
        cursor.execute("""
            SELECT 1 FROM Artist_Locations WHERE artist_id = %s AND location_id = %s
        """, (row['ConstituentID'], location_id))
        
        if not cursor.fetchone():
            cursor.execute("""
                INSERT INTO Artist_Locations (artist_id, location_id)
                VALUES (%s, %s)
            """, (row['ConstituentID'], location_id))

In [ ]:
# Convert non-list entries into a list
artists_combined['Influencedby'] = artists_combined['Influencedby'].apply(lambda x: x if isinstance(x, list) else [x] if pd.notna(x) else [])

# Iterate through rows and insert into Influences table
for _, row in artists_combined.iterrows():
    # Check if 'Influencedby' column is empty or contains "Unknown" or NaN
    if not row['Influencedby'] or "Unknown" in row['Influencedby']:
        continue  # Skip this row if it has nulls or "Unknown"
    
    # Iterate through Influencedby and insert into Influences
    for influencer in row['Influencedby']:
        if pd.isna(influencer) or influencer == "NaN" or influencer == "Unknown":
            continue  # Skip invalid influencer values
        
        # Get influencer's ID from Artists table
        cursor.execute("SELECT artist_id FROM Artists WHERE display_name = %s", (influencer,))
        influencer_id = cursor.fetchone()
        
        if influencer_id:
            influencer_id = influencer_id[0]  # Extract influencer_id
            
            # Insert into Influences table
            cursor.execute("""
                INSERT INTO Influences (influencer_id, influenced_id)
                VALUES (%s, %s)
            """, (influencer_id, row['ConstituentID']))

In [ ]:
# Convert non-list entries into list
artists_combined['FriendsandCoworkers'] = artists_combined['FriendsandCoworkers'].apply(
    lambda x: x if isinstance(x, list) else [x] if pd.notna(x) else []
)

# Iterate through rows and insert into Friends table
for _, row in artists_combined.iterrows():
    # Check if 'FriendsandCoworkers' column is empty or contains NaN/invalid values
    if not row['FriendsandCoworkers'] or "Unknown" in row['FriendsandCoworkers']:
        continue  # Skip if no friends/coworkers or contains "Unknown"
    
    # Iterate through Friends and insert into Friends table
    for friend in row['FriendsandCoworkers']:
        if pd.isna(friend) or friend == "NaN" or friend == "Unknown":
            continue  # Skip invalid friend values
        
        # Get artist_id of current artist
        artist_id = row['ConstituentID']
        
        # Get 'artist_id' of friend from Artists table
        cursor.execute("SELECT artist_id FROM Artists WHERE display_name = %s", (friend,))
        friend_id = cursor.fetchone()
        
        if friend_id:
            friend_id = friend_id[0]  # Extract friend's artist_id
            
            # Ensure pair is inserted only once
            if artist_id < friend_id:
                artist1_id, artist2_id = artist_id, friend_id
            else:
                artist1_id, artist2_id = friend_id, artist_id
            
            # Check if pair already exists in Friends table to avoid duplicates
            cursor.execute("""
                SELECT 1 FROM Friends WHERE artist1_id = %s AND artist2_id = %s
            """, (artist1_id, artist2_id))
            existing_friend = cursor.fetchone()
            
            if not existing_friend:  # Only insert if the pair doesn't already exist
                cursor.execute("""
                    INSERT INTO Friends (artist1_id, artist2_id)
                    VALUES (%s, %s)
                """, (artist1_id, artist2_id))

In [ ]:
import pandas as pd

# Convert non-list entries into list
painter_palette['PaintingsExhibitedAt'] = painter_palette['PaintingsExhibitedAt'].apply(lambda x: x if isinstance(x, list) else [x] if pd.notna(x) else [])

# Iterate through rows and insert into Exhibitions table
for _, row in painter_palette.iterrows():
    for exhibition in row['PaintingsExhibitedAt']:
        # Skip if exhibition "Unknown" or invalid
        if pd.isna(exhibition) or exhibition == "Unknown":
            continue

        # Get exhibition name, location, and year
        exhibition_name = exhibition.strip()
        location = row['locations']  # Ensure this column contains the correct exhibition location(s)
        year = row['FirstYear']  # Assuming 'FirstYear' column contains the year of exhibition
        painting_count = row['PaintingsExhibitedAtCount']
        
        # If location is NaN or None, skip
        if pd.isna(location) or location is None:
            continue

        # If multiple locations, choose the first
        location_id = None
        if isinstance(location, list):
            for loc in location:
                loc = loc.strip()
                cursor.execute("SELECT location_id FROM Locations WHERE name = %s", (loc,))
                location_id_result = cursor.fetchone()
                if location_id_result:
                    location_id = location_id_result[0]  # Get the first matching location_id
                    break
        else:
            location = location.strip()
            cursor.execute("SELECT location_id FROM Locations WHERE name = %s", (location,))
            location_id_result = cursor.fetchone()
            if location_id_result:
                location_id = location_id_result[0]  # Get location_id for the location

        # If no location found, skip
        if location_id is None:
            continue

        # Insert exhibition into the Exhibitions table
        cursor.execute("""
            INSERT INTO Exhibitions (exhibition_name, location_id, year, painting_count)
            VALUES (%s, %s, %s, %s)
        """, (exhibition_name, location_id, year, painting_count))

Commit and close the connection:

In [ ]:
# Commit changes
conn.commit()

# Close the connection
cursor.close()
conn.close()